In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math
import glob
import cv2

In [ ]:
#loading_data
from google.colab import files
uploaded=files.upload()

Saving Preprocessed_data_08-19.csv to Preprocessed_data_08-19.csv


In [ ]:
# url = "https://raw.githubusercontent.com/arjunparmar/IPL-2020/master/Arjun/final_08-19.csv"
# data = pd.read_csv(url)

In [ ]:
data = pd.read_csv("Preprocessed_data_08-19.csv")
display(data.tail())
data.head()


,team_A,team_B,toss_winner,top_batsmen_points_A,top_batsmen_points_B,top_baller_points_A,top_baller_points_B,match_won_after_winning_toss_A,match_won_after_winning_toss_B,runs_in_powerplay_A,runs_in_powerplay_B,run_rate_A,run_rate_B,most_value_player_points_A,most_value_player_points_B,win_ratio_A,win_ratio_B,ground_A,ground_B,winner
751,4,7,1,0.4175,0.4850,0.325359,0.747608,0.563218,0.555556,0.480000,0.457143,0.028,0.421,0.788321,0.865485,0.22,0.78,0,1,1.0
752,0,7,0,0.5900,0.4850,0.677033,0.747608,0.649351,0.555556,0.240000,0.447619,0.131,0.421,0.796663,0.865485,0.46,0.54,1,0,1.0
753,12,1,1,0.4700,0.2875,0.630383,0.808612,0.500000,0.000000,0.520000,0.533333,0.577,0.044,0.724192,0.705944,0.67,0.33,0,0,1.0
754,1,0,1,0.2875,0.5900,0.808612,0.677033,0.000000,0.649351,0.360000,0.400000,0.044,0.131,0.705944,0.796663,0.33,0.67,0,0,1.0
755,7,0,0,0.4850,0.5900,0.747608,0.677033,0.555556,0.649351,0.413333,0.514286,0.421,0.131,0.865485,0.796663,0.54,0.46,0,0,0.0


,team_A,team_B,toss_winner,top_batsmen_points_A,top_batsmen_points_B,top_baller_points_A,top_baller_points_B,match_won_after_winning_toss_A,match_won_after_winning_toss_B,runs_in_powerplay_A,runs_in_powerplay_B,run_rate_A,run_rate_B,most_value_player_points_A,most_value_player_points_B,win_ratio_A,win_ratio_B,ground_A,ground_B,winner
0,4,9,1,0.2850,0.2175,0.712919,0.309809,0.0,0.0,0.613333,0.247619,-0.147,-1.160,0.536496,0.539625,0.0,0.0,0,1,0.0
1,0,6,0,0.9625,0.9550,0.514354,0.483254,0.0,0.0,0.506667,0.600000,-0.192,0.509,0.709072,0.758081,0.0,0.0,0,0,0.0
2,11,1,0,0.5500,0.2850,0.522727,0.545455,0.0,0.0,0.333333,0.523810,0.632,0.342,0.956204,0.539625,0.0,0.0,0,0,1.0
3,7,9,0,0.3350,0.2175,1.000000,0.309809,0.0,0.0,0.426667,0.380952,0.570,-1.160,0.754953,0.539625,0.0,0.0,1,0,1.0
4,2,4,0,0.4875,0.2850,0.561005,0.712919,0.0,0.0,0.320000,0.247619,-0.467,-0.147,0.623566,0.536496,0.0,0.0,0,1,1.0


In [ ]:
#spliting_data

display(data.tail())

print(len(data))
#shuffling_data
#p = np.random.permutation(len(data))
#print(p)
#data = data.loc[p]

train_data = data[: 696]

test_data = data[696 :]  

#Hyd = 0
#Mum = 1
#Guj = 2
#Pune= 3
#Bangalore = 4
#Kolk = 5
#Delhi = 6
#Punjab = 7
#kerla = 10
#chennai = 8
#Rajas  = 9

,team_A,team_B,toss_winner,top_batsmen_points_A,top_batsmen_points_B,top_baller_points_A,top_baller_points_B,match_won_after_winning_toss_A,match_won_after_winning_toss_B,runs_in_powerplay_A,runs_in_powerplay_B,run_rate_A,run_rate_B,most_value_player_points_A,most_value_player_points_B,win_ratio_A,win_ratio_B,ground_A,ground_B,winner
751,4,7,1,0.4175,0.4850,0.325359,0.747608,0.563218,0.555556,0.480000,0.457143,0.028,0.421,0.788321,0.865485,0.22,0.78,0,1,1.0
752,0,7,0,0.5900,0.4850,0.677033,0.747608,0.649351,0.555556,0.240000,0.447619,0.131,0.421,0.796663,0.865485,0.46,0.54,1,0,1.0
753,12,1,1,0.4700,0.2875,0.630383,0.808612,0.500000,0.000000,0.520000,0.533333,0.577,0.044,0.724192,0.705944,0.67,0.33,0,0,1.0
754,1,0,1,0.2875,0.5900,0.808612,0.677033,0.000000,0.649351,0.360000,0.400000,0.044,0.131,0.705944,0.796663,0.33,0.67,0,0,1.0
755,7,0,0,0.4850,0.5900,0.747608,0.677033,0.555556,0.649351,0.413333,0.514286,0.421,0.131,0.865485,0.796663,0.54,0.46,0,0,0.0


756


In [ ]:
X_train = np.array(train_data.drop(["winner"] , axis = 1).copy()).T
Y_train = np.array([train_data['winner'].copy()])

X_test = np.array(test_data.drop(["winner"] , axis = 1).copy()).T
Y_test = np.array([test_data['winner'].copy()])

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(19, 696)
(1, 696)
(19, 60)
(1, 60)


In [ ]:
#Data_preprocesing
shape = (Y_train.max()+1, Y_train.shape[1]) #Generating one_hot_vector so that later we can apply multi class classification
one_hot = np.zeros(shape)
rows= np.arange(Y_train.shape[1])
one_hot[Y_train , rows]=1
Y_train = one_hot

In [ ]:
#Data_preprocesing
shape = (Y_test.max()+1, Y_test.shape[1]) #Generating one_hot_vector so that later we can apply multi class classification
one_hot = np.zeros(shape)
rows= np.arange(Y_test.shape[1])
one_hot[Y_test , rows]=1
Y_test = one_hot

In [ ]:
Y_test = np.vstack([Y_test,np.zeros((1,60))]) #Adding kerala on the last row in 2019 as it was not playing

In [ ]:
print(Y_train.shape)
print(Y_test.shape)

(11, 696)
(11, 60)


In [ ]:
X_train = np.copy(X_train).astype(float)
X_test  = np.copy(X_test).astype(float)
#data_preprocessing
mean = np.array([np.mean(X_train , axis = 1)]).T  #normalizing the training set
max = np.array([np.max(X_train , axis = 1)]).T
min = np.array([np.min(X_train , axis=1)]).T
#variance_train = np.mean((X_train - mean)**2)
#X_train = ((X_train - mean) / np.sqrt(variance_train))
X_train = ((X_train - mean) / (max - min))

print(X_train.shape)
print(Y_train.shape)

mean = np.array([np.mean(X_test , axis = 1)]).T  #normalizing the test set
max = np.array([np.max(X_test , axis = 1)]).T
min = np.array([np.min(X_test , axis=1)]).T
#variance_test = np.mean((X_test - mean)**2)
#X_test = ((X_test - mean) / np.sqrt(variance_test))
X_test = ((X_test - mean) / (max - min))

print(X_test.shape)
print(Y_test.shape)

(8, 696)
(11, 696)
(8, 60)
(11, 60)


In [ ]:
n_X=X_train.shape[0]
m=X_train.shape[1]

def softmax(A):               #Softmax gives a good classification as it standardizes the output values.
    A = A - np.max(A)
    return np.exp(A) / np.sum(np.exp(A),axis=0) 

nums = np.array([4, 5, 6])
print(softmax(nums))

[0.09003057 0.24472847 0.66524096]


In [ ]:
#initialize_parameters
layer_dims=[19,100,80,60,30,15,13,1]                               #use this for He initializtion *np.sqrt(2/layer_dims[l-1])
                                                                       
parameters={}
L=len(layer_dims)
for l in range(1,L):
  parameters["W"+str(l)]= np.random.randn(layer_dims[l],layer_dims[l-1])*np.sqrt(2/layer_dims[l-1])

  parameters["b"+str(l)]= np.zeros((layer_dims[l],1))*0.01


def DNN_model(X,Y,parameters,layer_dims,num_iterations,learning_rate,lambd):

  m=X.shape[1]
  Y_prediction=np.zeros((1,m))
  forward_cache={}
  
  forward_cache["A0"]=np.copy(X)
  L=len(layer_dims)
  grads={}
  costs=[]
  cost_1=1000
  cost_2=1000
  reg_cost = 0
  tot_reg_cost = 0
  for i in range(0,num_iterations):
    
    A=np.copy(X)
    for l in range(1,L-1):
      
      A_prev=np.copy(A)
      
      forward_cache["Z"+str(l)]=np.dot(parameters["W"+str(l)],A_prev)+parameters["b"+str(l)]
      A=np.maximum(0,forward_cache["Z"+str(l)])
      
      forward_cache["A"+str(l)]=np.copy(A)
      #Regulariztion_cost
      reg_cost += np.sum(np.square(parameters["W" + str(l)]))


    #last_layer
    tot_reg_cost = (1/m)*(lambd/2)*(reg_cost + np.sum(np.square(parameters["W"+str(L-1)])))
    W=parameters["W"+str(L-1)]
    b=parameters["b"+str(L-1)]
    forward_cache["Z"+str(L-1)]=np.dot(W,A)+b
    AL=  softmax(forward_cache["Z"+str(L-1)])

    #Regulariztion_cost
    

    #compute_cost
    m=Y.shape[1]
    cost_1 = -np.mean(Y * np.log(AL + 1e-8)) + tot_reg_cost

    #print(np.sum(W))
    #print(np.sum(b))
    print(cost_1)

    costs.append(float(cost_1))
    #backward_prop
    grads["dZ"+str(L-1)]=AL-Y
    grads["dW"+str(L-1)]=(1/m)*(np.dot(grads["dZ"+str(L-1)],forward_cache["A"+str(L-2)].T)) + (lambd/m)*(parameters["W"+str(L-1)])
    grads["db"+str(L-1)]=(1/m)*(np.sum(grads["dZ"+str(L-1)],axis=1,keepdims=True))
    for l in reversed(range(1,L-1)):
      grads["dZ"+str(l)]=np.multiply(np.dot(parameters["W"+str(l+1)].T,grads["dZ"+str(l+1)]),np.int64(forward_cache["A"+str(l)]>0))
      grads["dW"+str(l)]=(1/m)*(np.dot(grads["dZ"+str(l)],(forward_cache["A"+str(l-1)]).T)) + (lambd/m)*(parameters["W"+str(l)])
      grads["db"+str(l)]=(1/m)*(np.sum(grads["dZ"+str(l)],axis=1,keepdims=True))
    for l in range(1, L):
      parameters["W" + str(l)] = parameters["W" + str(l)] - learning_rate * grads["dW" + str(l)]
      parameters["b" + str(l)] = parameters["b" + str(l)] - learning_rate * grads["db" + str(l)]
     
  return Y_prediction,parameters,costs,AL

In [ ]:
Y_prediction,parameters,costs,AL= DNN_model(X_train,Y_train,parameters,layer_dims,num_iterations=5000,learning_rate=0.01,lambd=0)
num_iterations=5000

plt.plot(list(range(num_iterations)),list(costs))
plt.show()

In [ ]:
print(AL.shape)
print(Y_train.shape)

(11, 696)
(11, 696)


In [ ]:
def DNN_model_batch(X,Y,parameters):

  m=X.shape[1]
  Y_prediction=np.zeros((1,m))
  forward_cache={}
  
  forward_cache["A0"]=np.copy(X)
  L=len(layer_dims)
  grads={}
  costs=[]
  cost_1=1000
  cost_2=1000
  for i in range(0,num_iterations):
    
    A=np.copy(X)
    for l in range(1,L-1):
      
      A_prev=np.copy(A)
      
      forward_cache["Z"+str(l)]=np.dot(parameters["W"+str(l)],A_prev)+parameters["b"+str(l)]
      A=np.maximum(0,forward_cache["Z"+str(l)])
      
      forward_cache["A"+str(l)]=np.copy(A)
    #last_layer
    W=parameters["W"+str(L-1)]
    b=parameters["b"+str(L-1)]
    forward_cache["Z"+str(L-1)]=np.dot(W,A)+b
    AL_train=  softmax(forward_cache["Z"+str(L-1)])
     
  return parameters,AL_train

In [ ]:
parameters,AL_train= DNN_model_batch(X_train,Y_train,parameters)

In [ ]:
  for i in range(AL_train.shape[1]):
    
    a = np.max(AL_train[:,i])
    AL_train[:,i] = (AL_train[:,i]==a)
  fai = np.absolute(AL_train - Y_train)
  count = 0
  for i in range(0,fai.shape[1]):
    if np.sum(fai[:,i]) == 0:
      count +=1
  print(count)
  print((count /X_train.shape[1])*100)

In [ ]:
#for test set
parameters,AL_test= DNN_model_batch(X_test,Y_test,parameters)

In [ ]:
  for i in range(AL_test.shape[1]):
    
    a = np.max(AL_test[:,i])
    AL_test[:,i] = (AL_test[:,i]==a)
  fai = np.absolute(AL_test - Y_test)
  count = 0
  for i in range(0,fai.shape[1]):
    if np.sum(fai[:,i]) == 0:
      count +=1
  print(count)
  print((count /X_test.shape[1])*100)